In [1]:
from configparser import ConfigParser
import customtkinter
from tkinter import Canvas, ttk

class PreferenceView(customtkinter.CTkFrame):
    def __init__(self, parent, items: ConfigParser):
        super().__init__(master=parent)
        self.__items = items
        self.__toggler = {}

        canvas = Canvas(master=self)
        canvas.pack(side='left', fill="both", expand=True)

        canvas_scrollbar= ttk.Scrollbar(canvas, orient="vertical", command=canvas.yview)
        canvas_scrollbar.pack(side='right', fill="y")

        canvas.configure(yscrollcommand=canvas_scrollbar.set)

        canvas.bind('<Configure>', lambda e: canvas.configure(scrollregion = canvas.bbox('all')))

        list_frame = customtkinter.CTkFrame(canvas)
        canvas.create_window((0,0), window=list_frame, anchor="nw")

        list_frame.pack(fill="both", expand=True)

        for key in items.keys():
            self.__toggler[key] = {}
            self.__toggler[key]['toggle'] = False
            # Label and button frame
            header = customtkinter.CTkFrame(list_frame, height=45, fg_color=("white", "gray28"))
            # Items frame
            sub_frame = customtkinter.CTkFrame(list_frame, relief="sunken")
            header.pack(side="top", fill="x")
            sub_frame.pack(side="top", fill="both", expand=True)

            toggle_btn = customtkinter.CTkButton(master=header,
                                                    text="+",
                                                    text_font=("Roboto Medium", -16),
                                                    hover_color=None,
                                                    fg_color=None,  # <- no fg_color
                                                    command=lambda: self.__toggle(key))

            text_label = customtkinter.CTkLabel(header, text=key, text_font=("Roboto Medium", -17))

            text_label.place(anchor="nw")
            toggle_btn.place(relx=0.9, rely=0.5, anchor=customtkinter.CENTER)

            sub_frame.pack(side="top", fill="both", expand=True)

            for item in items[key]:
                customtkinter.CTkLabel(sub_frame, text=item).pack(side="top")

            self.__toggler[key]['subframe'] = sub_frame
            self.__toggler[key]['button'] = toggle_btn
            self.__toggle(key)


    def __toggle(self, key):
        if self.__toggler[key]['toggle']:
            self.__toggler[key]['subframe'].pack(side="top", fill="x", expand=True)
            self.__toggler[key]['button'].configure(text='-')
            self.__toggler[key]['toggle'] = False
        else:
            self.__toggler[key]['subframe'].forget()
            self.__toggler[key]['button'].configure(text='+')
            self.__toggler[key]['toggle'] = True

In [2]:
import os
import signal
from turtle import right
import customtkinter
import tkinter
from PIL import Image
from pystray import Icon
from pystray import MenuItem as item
from win10toast_click import ToastNotifier
from config.config import Config
from data.client import ClientData
from tkinter import ttk
from tkinter import Canvas

class ViewWindow(customtkinter.CTkToplevel):
    WINDOW_WIDTH = 780
    WINDOW_HEIGHT = 520
    EXIT_WIDTH = 320
    EXIT_HEIGHT = 130

    def __init__(self, parent: customtkinter.CTk, config: Config, debug: bool = False) -> None:
        super().__init__(master=parent)
        self.__parent = parent
        self.__parent.withdraw()
        self.__config = config
        self.__client_data = ClientData(config, debug)
        self.__client_data.load_client_data()

        # Set icon path and icon for the application
        self.icon_path = os.path.join('extra', 'imgs', 'favicon.ico')
        self.iconbitmap(self.icon_path)

        # Setup the dimension and position
        ws = self.winfo_screenwidth() # width of the screen
        hs = self.winfo_screenheight() # height of the screen
        # calculate x and y coordinates for the Tk root window
        self.main_x = int((ws/2) - (ViewWindow.WINDOW_WIDTH/2))
        self.main_y = int((hs/2) - (ViewWindow.WINDOW_HEIGHT/2))
        self.geometry(f"{ViewWindow.WINDOW_WIDTH}x{ViewWindow.WINDOW_HEIGHT}+{self.main_x}+{self.main_y}")

        # Specify the delete window protocol with custom function/dialog
        self.protocol("WM_DELETE_WINDOW", self.__custom_dialog)

        # Title of the app
        self.title("Lotro Data")

        # ============ create two frames ============

        # configure grid layout (2x1)
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)

        self.frame_left = customtkinter.CTkFrame(master=self,
                                                 width=180,
                                                 corner_radius=0)
        self.frame_left.grid(row=0, column=0, sticky="nswe")

        self.frame_right = customtkinter.CTkFrame(master=self)
        self.frame_right.grid(row=0, column=1, sticky="nswe", padx=20, pady=20)

        # ============ frame_left ============

        # configure grid layout (1x11)
        self.frame_left.grid_rowconfigure(0, minsize=10)   # empty row with minsize as spacing
        self.frame_left.grid_rowconfigure(5, weight=1)  # empty row as spacing
        self.frame_left.grid_rowconfigure(8, weight=1)    # empty row with minsize as spacing
        self.frame_left.grid_rowconfigure(11, weight=1)  # empty row with minsize as spacing

        self.frame_title = customtkinter.CTkLabel(master=self.frame_left,
                                              text="Lotro Data\nExtractor",
                                              text_font=("Roboto Medium", -20))  # font name and size in px
        self.frame_title.grid(row=1, column=0, pady=10, padx=10)

        # self.button_1 = customtkinter.CTkButton(master=self.frame_left,
        #                                         text="Button1",
        #                                         command=self.button_event)
        # self.button_1.grid(row=2, column=0, pady=10, padx=20)

        # ============ frame_right ============
        

        pref_frame = PreferenceView(parent=self.frame_right, items=self.__client_data.pref)
        pref_frame.pack(side="top", fill="x")


        
        # self.button_2 = customtkinter.CTkButton(master=self.frame_left,
        #                                         text="Button2",
        #                                         command=self.button_event)
        # self.button_2.grid(row=3, column=0, pady=10, padx=20)

        # self.button_3 = customtkinter.CTkButton(master=self.frame_left,
        #                                         text="Button3",
        #                                         command=self.button_event)
        # self.button_3.grid(row=4, column=0, pady=10, padx=20)

        # self.label_mode = customtkinter.CTkLabel(master=self.frame_left, text="Appearance Mode:")
        # self.label_mode.grid(row=9, column=0, pady=0, padx=20, sticky="w")

        # self.optionmenu_1 = customtkinter.CTkOptionMenu(master=self.frame_left,
        #                                                 values=["Light", "Dark", "System"],
        #                                                 command=self.change_appearance_mode)
        # self.optionmenu_1.grid(row=10, column=0, pady=10, padx=20, sticky="w")


    """
    Function to destroy the windows called from the confirmation window.
    :param popup: The popup to destroy.
    :type popup: customtkinter.CTKTopevel
    :return None
    """
    def __quit_app(self, popup: customtkinter.CTkToplevel = None) -> None:
        if popup is not None:
            popup.grab_release()
            popup.destroy()       
        self.__config.close_mem() 
        os.kill(os.getpid(), signal.SIGTERM)



    """
    Function to destroy the window and the icon called from the icon menu.
    :param icon: The icon to destroy.
    :type icon: pystray.Icon
    :return None
    """
    def __quit_both_app(self, icon: Icon = None) -> None:
        if icon is not None:
            icon.stop()
        self.__config.close_mem() 
        os.kill(os.getpid(), signal.SIGTERM)


    """
    Function to show the app window and destroy the icon if open.
    :param icon: The icon to destroy.
    :type icon: pystray.Icon
    :return None
    """
    def __show_app(self, icon) -> None:
        if icon is not None:
            icon.stop()
        self.after(0, self.deiconify)


    """
    Function to withdraw the app window, destroy popup, and create the icon in the system tray.
    :param popup: The popup to destroy.
    :type popup: customtkinter.CTKTopevel
    :return None
    """
    def __withdraw_app(self, popup: customtkinter.CTkToplevel = None) -> None:  
        # Release the focus and destroy the popup
        popup.grab_release()
        popup.destroy()

        # Withdraw the app itself and send a notification
        self.withdraw()
        toaster = ToastNotifier()
        toaster.show_toast("Lotro Extractor",
                   "Lotro Extractor is running in the system tray.",
                   icon_path=self.icon_path,
                   duration=3, threaded=True)

        # Create a system tray icon and run the icon
        image = Image.open(self.icon_path)
        menu = (item('Show', action=self.__show_app), item('Quit', action=self.__quit_both_app))
        self.icon = Icon("Lotro Extractor", image, "Lotro Extractor", menu)
        self.icon.run()
        

    def __custom_dialog(self) -> None:
        # Create a top level dialog
        popout = customtkinter.CTkToplevel(self)

        # Placement of the dialog
        # calculate x and y coordinates for the dialog window
        popout_x = int((self.main_x+ViewWindow.WINDOW_WIDTH/2) - (ViewWindow.EXIT_WIDTH/2))
        popout_y = int((self.main_y+ViewWindow.WINDOW_HEIGHT/2) - (ViewWindow.EXIT_HEIGHT/2))
        popout.geometry(f"{ViewWindow.EXIT_WIDTH}x{ViewWindow.EXIT_HEIGHT}+{popout_x}+{popout_y}")
        # Disable resize
        popout.resizable(False, False)

        # Grab the focus in the dialog window
        popout.grab_set()

        # Set the title and the information
        popout.title("Confirmation")
        label = customtkinter.CTkLabel( popout, text="Proceed to close or minimize application?")
        label.pack(pady=20)
        minimize_button = customtkinter.CTkButton(master=popout, text="Minimize", command=lambda: self.__withdraw_app(popout))
        minimize_button.place(relx=0.27, rely=0.5, anchor=tkinter.CENTER)
        quit_button = customtkinter.CTkButton(master=popout, text="Quit", command=lambda: self.__quit_app(popup=popout))
        quit_button.place(relx=0.73, rely=0.5, anchor=tkinter.CENTER)
        

In [3]:
import os
import signal
import threading
import time
import tkinter

import customtkinter
import psutil
from config.config import Config
from PIL import Image
from pystray import Icon
from pystray import MenuItem as item
from win10toast_click import ToastNotifier

class App(customtkinter.CTk):
    MAIN_WIDTH = 300
    MAIN_HEIGHT = 100
    EXIT_WIDTH = 320
    EXIT_HEIGHT = 130

    """
    Init method to initialize the application.
    :param None
    :return None
    """
    def __init__(self, debug: bool = False) -> None:
        super().__init__()
        self.__debug = debug

        # =============== Define the main window and appearance ===============
        customtkinter.set_appearance_mode("System")  # Modes: system (default), light, dark
        customtkinter.set_default_color_theme("blue")  # Themes: blue (default), dark-blue, green

        # Set icon path and icon for the application
        self.icon_path = os.path.join('extra', 'imgs', 'favicon.ico')
        self.iconbitmap(self.icon_path)

        # Specify the delete window protocol with custom function/dialog
        self.protocol("WM_DELETE_WINDOW", self.__custom_dialog)

        # Title of the app
        self.title("Lotro Extractor")

        # Placement of the window
        # get screen width and height
        ws = self.winfo_screenwidth() # width of the screen
        hs = self.winfo_screenheight() # height of the screen
        # calculate x and y coordinates for the Tk root window
        self.main_x = int((ws/2) - (App.MAIN_WIDTH/2))
        self.main_y = int((hs/2) - (App.MAIN_HEIGHT/2))
        self.geometry(f"{App.MAIN_WIDTH}x{App.MAIN_HEIGHT}+{self.main_x}+{self.main_y}")

        # =============== 
        self.waiting_label = customtkinter.CTkLabel(master=self, text="Waiting for lotro client to start...")
        self.waiting_label.place(relx=0.5, rely=0.5, anchor=tkinter.CENTER)
        
        # =============== Threaded wait for client ==============
        threading.Thread(target=self.__run_config, args=[self.__debug]).start()



    def __run_config(self, debug: bool = False) -> None:
        config = Config(debug=debug)
        if config.await_process():
            threading.Thread(target=self.__monitor_process, args=[config]).start()
            config.set_address()
            App.MAIN_WIDTH = App.MAIN_WIDTH+100
            self.geometry(f"{App.MAIN_WIDTH}x{App.MAIN_HEIGHT}")
            self.waiting_label.configure(text=f"Client found at {config.lotro_exe} with PID: {config.pid}! Retrieving info..")
            if not debug: time.sleep(10)
            ViewWindow(self, config, debug)



    def __monitor_process(self, config: Config):
        """ Check if the client is still running with given pid. """
        while(psutil.pid_exists(config.pid)):
            time.sleep(1)
        config.close_mem()
        os.kill(os.getpid(), signal.SIGTERM)

    """
    Function to destroy the windows called from the confirmation window.
    :param popup: The popup to destroy.
    :type popup: customtkinter.CTKTopevel
    :return None
    """
    def __quit_app(self, popup: customtkinter.CTkToplevel = None) -> None:
        if popup is not None:
            popup.grab_release()
            popup.destroy()        
        os.kill(os.getpid(), signal.SIGTERM)



    """
    Function to destroy the window and the icon called from the icon menu.
    :param icon: The icon to destroy.
    :type icon: pystray.Icon
    :return None
    """
    def __quit_both_app(self, icon: Icon = None) -> None:
        if icon is not None:
            icon.stop()
        os.kill(os.getpid(), signal.SIGTERM)


    """
    Function to show the app window and destroy the icon if open.
    :param icon: The icon to destroy.
    :type icon: pystray.Icon
    :return None
    """
    def __show_app(self, icon) -> None:
        if icon is not None:
            icon.stop()
        self.after(0, self.deiconify)


    """
    Function to withdraw the app window, destroy popup, and create the icon in the system tray.
    :param popup: The popup to destroy.
    :type popup: customtkinter.CTKTopevel
    :return None
    """
    def __withdraw_app(self, popup: customtkinter.CTkToplevel = None) -> None:  
        # Release the focus and destroy the popup
        popup.grab_release()
        popup.destroy()

        # Withdraw the app itself and send a notification
        self.withdraw()
        toaster = ToastNotifier()
        toaster.show_toast("Lotro Extractor",
                   "Lotro Extractor is running in the system tray.",
                   icon_path=self.icon_path,
                   duration=3, threaded=True)

        # Create a system tray icon and run the icon
        image = Image.open(self.icon_path)
        menu = (item('Show', action=self.__show_app), item('Quit', action=self.__quit_both_app))
        self.icon = Icon("Lotro Extractor", image, "Lotro Extractor", menu)
        self.icon.run()


    """
    Function to create a custom dialog window.
    :param None
    :return None
    """
    def __custom_dialog(self) -> None:
        # Create a top level dialog
        popout = customtkinter.CTkToplevel(self)

        # Placement of the dialog
        # calculate x and y coordinates for the dialog window
        popout_x = int((self.main_x+App.MAIN_WIDTH/2) - (App.EXIT_WIDTH/2))
        popout_y = int((self.main_y+App.MAIN_HEIGHT/2) - (App.EXIT_HEIGHT/2))
        popout.geometry(f"{App.EXIT_WIDTH}x{App.EXIT_HEIGHT}+{popout_x}+{popout_y}")
        # Disable resize
        popout.resizable(False, False)

        # Grab the focus in the dialog window
        popout.grab_set()

        # Set the title and the information
        popout.title("Confirmation")
        label = customtkinter.CTkLabel( popout, text="Proceed to close or minimize application?")
        label.pack(pady=20)
        minimize_button = customtkinter.CTkButton(master=popout, text="Minimize", command=lambda: self.__withdraw_app(popout))
        minimize_button.place(relx=0.27, rely=0.5, anchor=tkinter.CENTER)
        quit_button = customtkinter.CTkButton(master=popout, text="Quit", command=lambda: self.__quit_app(popup=popout))
        quit_button.place(relx=0.73, rely=0.5, anchor=tkinter.CENTER)


In [4]:
app = App(debug=True)
app.mainloop()

Retrieving information from the client process...
     Client: lotroclient.exe
     Id: 25032
Lotro Client: C:\Program Files (x86)\Steam\steamapps\common\Lord of the Rings Online\lotroclient.exe
    Base Address: 4194304
    64-bits? False
    Pointer Size: 4
    Int Size: 4
    Map Int Size: 4
    Bucket Size: 24
    World Entity Offset: 16
Entities Table Offset: 1614695
References Table Offset: 268473
Client/Account Data Offset: 48018
Storage Data Offset: 17581637
Client Server: [EN] Evernight
    Language: English
    Pref Path: C:\Users\s2665220\Documents\The Lord of the Rings Online\UserPreferences.ini


: 

: 

In [ ]:
# BACKUP
import customtkinter
from tkinter import ttk

class CustomFrame(customtkinter.CTkFrame):
    def __init__(self, parent, canvas, name, label, items):
        super().__init__(master=parent)
        self.__show = False
        self.__name = name

        # Label and button frame
        header = customtkinter.CTkFrame(self, height=45, fg_color=("white", "gray28"))
        # Items frame
        self.sub_frame = customtkinter.CTkFrame(self, relief="sunken")
        header.pack(side="top", fill="x")
        self.sub_frame.pack(side="top", fill="both", expand=True)

        self.toggle_btn = customtkinter.CTkButton(master=header,
                                                text="+",
                                                text_font=("Roboto Medium", -16),
                                                hover_color=None,
                                                fg_color=None,  # <- no fg_color
                                                command=self.__toggle)

        self.text_label = customtkinter.CTkLabel(header, text=label, text_font=("Roboto Medium", -17))

        self.bind('<Configure>', lambda e: canvas.configure(scrollregion = canvas.bbox('all')))

        self.text_label.place(anchor="nw")
        self.toggle_btn.place(relx=0.9, rely=0.5, anchor=customtkinter.CENTER)

        self.sub_frame.pack(side="top", fill="both", expand=True)

        for item in items:
            customtkinter.CTkLabel(self.sub_frame, text=item).pack(side="top")

        self.__toggle()

    def __toggle(self):
        if self.__show:
            self.sub_frame.pack(side="top", fill="x", expand=True)
            self.toggle_btn.configure(text='-')
            self.__show = False
        else:
            self.sub_frame.forget()
            self.toggle_btn.configure(text='+')
            self.__show = True

    def matches_name(self, name: str) -> bool:
        return name in self.__name

In [ ]:
# BACKUP
import os
import signal
from turtle import right
import customtkinter
import tkinter
from PIL import Image
from pystray import Icon
from pystray import MenuItem as item
from win10toast_click import ToastNotifier
from app.drop_down import CustomFrame
from config.config import Config
from data.client import ClientData
from tkinter import ttk
from tkinter import Canvas

class ViewWindow(customtkinter.CTkToplevel):
    WINDOW_WIDTH = 780
    WINDOW_HEIGHT = 520
    EXIT_WIDTH = 320
    EXIT_HEIGHT = 130

    def __init__(self, parent: customtkinter.CTk, config: Config, debug: bool = False) -> None:
        super().__init__(master=parent)
        self.__parent = parent
        self.__parent.withdraw()
        self.__config = config
        self.__client_data = ClientData(config, debug)
        self.__client_data.load_client_data()

        # Set icon path and icon for the application
        self.icon_path = os.path.join('extra', 'imgs', 'favicon.ico')
        self.iconbitmap(self.icon_path)

        # Setup the dimension and position
        ws = self.winfo_screenwidth() # width of the screen
        hs = self.winfo_screenheight() # height of the screen
        # calculate x and y coordinates for the Tk root window
        self.main_x = int((ws/2) - (ViewWindow.WINDOW_WIDTH/2))
        self.main_y = int((hs/2) - (ViewWindow.WINDOW_HEIGHT/2))
        self.geometry(f"{ViewWindow.WINDOW_WIDTH}x{ViewWindow.WINDOW_HEIGHT}+{self.main_x}+{self.main_y}")

        # Specify the delete window protocol with custom function/dialog
        self.protocol("WM_DELETE_WINDOW", self.__custom_dialog)

        # Title of the app
        self.title("Lotro Data")

        # ============ create two frames ============

        # configure grid layout (2x1)
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)

        self.frame_left = customtkinter.CTkFrame(master=self,
                                                 width=180,
                                                 corner_radius=0)
        self.frame_left.grid(row=0, column=0, sticky="nswe")

        self.frame_right = customtkinter.CTkFrame(master=self)
        self.frame_right.grid(row=0, column=1, sticky="nswe", padx=20, pady=20)

        # ============ frame_left ============

        # configure grid layout (1x11)
        self.frame_left.grid_rowconfigure(0, minsize=10)   # empty row with minsize as spacing
        self.frame_left.grid_rowconfigure(5, weight=1)  # empty row as spacing
        self.frame_left.grid_rowconfigure(8, weight=1)    # empty row with minsize as spacing
        self.frame_left.grid_rowconfigure(11, weight=1)  # empty row with minsize as spacing

        self.frame_title = customtkinter.CTkLabel(master=self.frame_left,
                                              text="Lotro Data\nExtractor",
                                              text_font=("Roboto Medium", -20))  # font name and size in px
        self.frame_title.grid(row=1, column=0, pady=10, padx=10)

        # self.button_1 = customtkinter.CTkButton(master=self.frame_left,
        #                                         text="Button1",
        #                                         command=self.button_event)
        # self.button_1.grid(row=2, column=0, pady=10, padx=20)

        # ============ frame_right ============
        right_canvas = Canvas(master=self.frame_right)
        right_canvas.pack(side='left', fill="both", expand=True)

        right_canvas_scrollbar= ttk.Scrollbar(right_canvas ,orient="vertical", command=right_canvas.yview)
        right_canvas_scrollbar.pack(side='right', fill="y")

        right_canvas.configure(yscrollcommand=right_canvas_scrollbar.set)

        right_canvas.bind('<Configure>', lambda e: right_canvas.configure(scrollregion = right_canvas.bbox('all')))

        list_frame = customtkinter.CTkFrame(right_canvas)
        right_canvas.create_window((0,0), window=list_frame, anchor="nw")

        key_val_map = {}
        for key in self.__client_data.pref.keys():
            pref_frame = CustomFrame(parent=list_frame, canvas=right_canvas, name=key, label=key, items=list(self.__client_data.pref[key].keys()))
            pref_frame.pack(side="top", fill="x")
            pref_list.append(pref_frame)

        list_frame.pack(fill="both", expand=True)

        
        # self.button_2 = customtkinter.CTkButton(master=self.frame_left,
        #                                         text="Button2",
        #                                         command=self.button_event)
        # self.button_2.grid(row=3, column=0, pady=10, padx=20)

        # self.button_3 = customtkinter.CTkButton(master=self.frame_left,
        #                                         text="Button3",
        #                                         command=self.button_event)
        # self.button_3.grid(row=4, column=0, pady=10, padx=20)

        # self.label_mode = customtkinter.CTkLabel(master=self.frame_left, text="Appearance Mode:")
        # self.label_mode.grid(row=9, column=0, pady=0, padx=20, sticky="w")

        # self.optionmenu_1 = customtkinter.CTkOptionMenu(master=self.frame_left,
        #                                                 values=["Light", "Dark", "System"],
        #                                                 command=self.change_appearance_mode)
        # self.optionmenu_1.grid(row=10, column=0, pady=10, padx=20, sticky="w")


    """
    Function to destroy the windows called from the confirmation window.
    :param popup: The popup to destroy.
    :type popup: customtkinter.CTKTopevel
    :return None
    """
    def __quit_app(self, popup: customtkinter.CTkToplevel = None) -> None:
        if popup is not None:
            popup.grab_release()
            popup.destroy()       
        self.__config.close_mem() 
        os.kill(os.getpid(), signal.SIGTERM)



    """
    Function to destroy the window and the icon called from the icon menu.
    :param icon: The icon to destroy.
    :type icon: pystray.Icon
    :return None
    """
    def __quit_both_app(self, icon: Icon = None) -> None:
        if icon is not None:
            icon.stop()
        self.__config.close_mem() 
        os.kill(os.getpid(), signal.SIGTERM)


    """
    Function to show the app window and destroy the icon if open.
    :param icon: The icon to destroy.
    :type icon: pystray.Icon
    :return None
    """
    def __show_app(self, icon) -> None:
        if icon is not None:
            icon.stop()
        self.after(0, self.deiconify)


    """
    Function to withdraw the app window, destroy popup, and create the icon in the system tray.
    :param popup: The popup to destroy.
    :type popup: customtkinter.CTKTopevel
    :return None
    """
    def __withdraw_app(self, popup: customtkinter.CTkToplevel = None) -> None:  
        # Release the focus and destroy the popup
        popup.grab_release()
        popup.destroy()

        # Withdraw the app itself and send a notification
        self.withdraw()
        toaster = ToastNotifier()
        toaster.show_toast("Lotro Extractor",
                   "Lotro Extractor is running in the system tray.",
                   icon_path=self.icon_path,
                   duration=3, threaded=True)

        # Create a system tray icon and run the icon
        image = Image.open(self.icon_path)
        menu = (item('Show', action=self.__show_app), item('Quit', action=self.__quit_both_app))
        self.icon = Icon("Lotro Extractor", image, "Lotro Extractor", menu)
        self.icon.run()
        

    def __custom_dialog(self) -> None:
        # Create a top level dialog
        popout = customtkinter.CTkToplevel(self)

        # Placement of the dialog
        # calculate x and y coordinates for the dialog window
        popout_x = int((self.main_x+ViewWindow.WINDOW_WIDTH/2) - (ViewWindow.EXIT_WIDTH/2))
        popout_y = int((self.main_y+ViewWindow.WINDOW_HEIGHT/2) - (ViewWindow.EXIT_HEIGHT/2))
        popout.geometry(f"{ViewWindow.EXIT_WIDTH}x{ViewWindow.EXIT_HEIGHT}+{popout_x}+{popout_y}")
        # Disable resize
        popout.resizable(False, False)

        # Grab the focus in the dialog window
        popout.grab_set()

        # Set the title and the information
        popout.title("Confirmation")
        label = customtkinter.CTkLabel( popout, text="Proceed to close or minimize application?")
        label.pack(pady=20)
        minimize_button = customtkinter.CTkButton(master=popout, text="Minimize", command=lambda: self.__withdraw_app(popout))
        minimize_button.place(relx=0.27, rely=0.5, anchor=tkinter.CENTER)
        quit_button = customtkinter.CTkButton(master=popout, text="Quit", command=lambda: self.__quit_app(popup=popout))
        quit_button.place(relx=0.73, rely=0.5, anchor=tkinter.CENTER)
        

In [1]:
import tkinter as tk

class Scrollbar(tk.Canvas):
    '''
        A scrollbar is gridded as a sibling of what it's scrolling.
    '''

    def __init__(self, parent, orient='vertical', hideable=False, **kwargs):
        print('kwargs is', kwargs)

        '''
            kwargs is {
                'width': 17, 
                'command': <bound method YView.yview of 
                    <widgets.Text object .!canvas.!frame.!frame.!text>>}

            https://stackoverflow.com/questions/15411107
            You can use dict.pop:... delete an item in a dictionary only if the given key exists... not certain if key exists in the dictionary...

                mydict.pop("key", None)

            ...if the second argument, None is not given, KeyError is raised if the key is not in the dictionary. Providing the second argument prevents the conditional exception... the second argument to .pop() is what it returns if the key is not found. 
        '''

        self.command = kwargs.pop('command', None)
        print('self.command is', self.command)
        tk.Canvas.__init__(self, parent, **kwargs)

        self.orient = orient
        self.hideable = hideable

        self.new_start_y = 0
        self.new_start_x = 0
        self.first_y = 0
        self.first_x = 0

        self.slidercolor = 'steelblue'
        self.troughcolor = 'lightgray'

        self.config(bg=self.troughcolor, bd=0, highlightthickness=0)

        # coordinates are irrelevant; they will be recomputed
        #   in the 'set' method
        self.create_rectangle(
            0, 0, 1, 1, 
            fill=self.slidercolor, 
            width=2, # this is border width
            outline='teal', 
            tags=('slider',))
        self.bind('<ButtonPress-1>', self.move_on_click)

        self.bind('<ButtonPress-1>', self.start_scroll, add='+')
        self.bind('<B1-Motion>', self.move_on_scroll)
        self.bind('<ButtonRelease-1>', self.end_scroll)

    def set(self, lo, hi):
        '''
            For resizing & repositioning the slider. The hideable
            scrollbar portion is by Fredrik Lundh, one of Tkinter's authors.
        '''

        lo = float(lo)
        hi = float(hi)

        if self.hideable is True:
            if lo <= 0.0 and hi >= 1.0:
                self.grid_remove()
                return
            else:
                self.grid()

        height = self.winfo_height()
        width = self.winfo_width()

        if self.orient == 'vertical':
            x0 = 2
            y0 = max(int(height * lo), 0)
            x1 = width - 2
            y1 = min(int(height * hi), height)
        # This was the tricky part of making a horizontal scrollbar 
        #   when I already knew how to make a vertical one.
        #   You can't just change all the "height" to "width"
        #   and "y" to "x". You also have to reverse what x0 etc 
        #   are equal to, comparing code in if and elif. Till that was
        #   done, everything worked but the horizontal scrollbar's 
        #   slider moved up & down.
        elif self.orient == 'horizontal':
            x0 = max(int(width * lo), 0)
            y0 = 2
            x1 = min(int(width * hi), width)
            y1 = height

        self.coords('slider', x0, y0, x1, y1)
        self.x0 = x0
        self.y0 = y0
        self.x1 = x1
        self.y1 = y1

    def move_on_click(self, event):
        if self.orient == 'vertical':
            # don't scroll on click if mouse pointer is w/in slider
            y = event.y / self.winfo_height()
            if event.y < self.y0 or event.y > self.y1:
                self.command('moveto', y)
            # get starting position of a scrolling event
            else:
                self.first_y = event.y
        elif self.orient == 'horizontal':
            # do nothing if mouse pointer is w/in slider
            x = event.x / self.winfo_width()
            if event.x < self.x0 or event.x > self.x1:
                self.command('moveto', x)
            # get starting position of a scrolling event
            else:
                self.first_x = event.x

    def start_scroll(self, event):
        if self.orient == 'vertical':
            self.last_y = event.y 
            self.y_move_on_click = int(event.y - self.coords('slider')[1])
        elif self.orient == 'horizontal':
            self.last_x = event.x 
            self.x_move_on_click = int(event.x - self.coords('slider')[0])

    def end_scroll(self, event):
        if self.orient == 'vertical':
            self.new_start_y = event.y
        elif self.orient == 'horizontal':
            self.new_start_x = event.x

    def move_on_scroll(self, event):

        # Only scroll if the mouse moves a few pixels. This makes
        #   the click-in-trough work right even if the click smears
        #   a little. Otherwise, a perfectly motionless mouse click
        #   is the only way to get the trough click to work right.
        #   Setting jerkiness to 5 or more makes very sloppy trough
        #   clicking work, but then scrolling is not smooth. 3 is OK.

        jerkiness = 3

        if self.orient == 'vertical':
            if abs(event.y - self.last_y) < jerkiness:
                return
            # scroll the scrolled widget in proportion to mouse motion
            #   compute whether scrolling up or down
            delta = 1 if event.y > self.last_y else -1
            #   remember this location for the next time this is called
            self.last_y = event.y
            #   do the scroll
            self.command('scroll', delta, 'units')
            # afix slider to mouse pointer
            mouse_pos = event.y - self.first_y
            if self.new_start_y != 0:
                mouse_pos = event.y - self.y_move_on_click
            self.command('moveto', mouse_pos/self.winfo_height()) 
        elif self.orient == 'horizontal':
            if abs(event.x - self.last_x) < jerkiness:
                return
            # scroll the scrolled widget in proportion to mouse motion
            #   compute whether scrolling left or right
            delta = 1 if event.x > self.last_x else -1
            #   remember this location for the next time this is called
            self.last_x = event.x
            #   do the scroll
            self.command('scroll', delta, 'units')
            # afix slider to mouse pointer
            mouse_pos = event.x - self.first_x
            if self.new_start_x != 0:
                mouse_pos = event.x - self.x_move_on_click
            self.command('moveto', mouse_pos/self.winfo_width()) 

    def colorize(self):
        print('colorize')
        self.slidercolor = 'blue'
        self.troughcolor = 'bisque'
        self.config(bg=self.troughcolor)

In [4]:
if __name__ == '__main__':

    def resize_scrollbar():
        root.update_idletasks()  
        canvas.config(scrollregion=canvas.bbox('all')) 

    def resize_window():
        root.update_idletasks()
        page_x = content.winfo_reqwidth()
        page_y = content.winfo_reqheight()
        root.geometry('{}x{}'.format(page_x, page_y))

    root = tk.Tk()
    root.config(bg='yellow')
    root.grid_columnconfigure(0, weight=1)
    root.grid_rowconfigure(0, weight=1)
    root.grid_rowconfigure(1, weight=0)

    canvas = tk.Canvas(root, bg='tan')
    canvas.grid(column=0, row=0, sticky='news')

    content = tk.Frame(canvas)
    content.grid_columnconfigure(0, weight=1)
    content.grid_rowconfigure(0, weight=1)

    ysb_canv = Scrollbar(root, width=24, hideable=True, command=canvas.yview)
    xsb_canv = Scrollbar(root, height=24, hideable=True, command=canvas.xview, orient='horizontal')
    canvas.config(yscrollcommand=ysb_canv.set, xscrollcommand=xsb_canv.set)

    frame = tk.Frame(content)
    frame.grid_columnconfigure(0, weight=0)
    frame.grid_rowconfigure(0, weight=1)

    text = tk.Text(frame, bd=0)
    ysb_txt = Scrollbar(frame, width=17, command=text.yview)

    text.config(yscrollcommand=ysb_txt.set)

    space = tk.Frame(content, width=1200, height=500)

    ysb_canv.grid(column=1, row=0, sticky='ns')
    xsb_canv.grid(column=0, row=1, sticky='ew')
    frame.grid(column=0, row=0, sticky='news')
    text.grid(column=0, row=0)
    ysb_txt.grid(column=1, row=0, sticky='ns')
    space.grid(column=0, row=1)

    text.insert('end', 'test')

    canvas.create_window(0, 0, anchor='nw', window=content)

    resize_scrollbar()
    resize_window()

    root.mainloop()

kwargs is {'width': 24, 'command': <bound method YView.yview of <tkinter.Canvas object .!canvas>>}
self.command is <bound method YView.yview of <tkinter.Canvas object .!canvas>>
kwargs is {'height': 24, 'command': <bound method XView.xview of <tkinter.Canvas object .!canvas>>}
self.command is <bound method XView.xview of <tkinter.Canvas object .!canvas>>
kwargs is {'width': 17, 'command': <bound method YView.yview of <tkinter.Text object .!canvas.!frame.!frame.!text>>}
self.command is <bound method YView.yview of <tkinter.Text object .!canvas.!frame.!frame.!text>>


In [6]:
import tkinter as tk
import random

class Example(tk.Frame):
    def __init__(self, parent):
        tk.Frame.__init__(self, parent, background="bisque")
        self.canvas = tk.Canvas(self, width=400, height=400)

        # define a larger virtual canvas 
        self.canvas.configure(scrollregion=(-400,-400,400,400), 
                              borderwidth=0, highlightthickness=0,
                              xscrollincrement=1, yscrollincrement=1)

        # arrange the widgets on the screen
        self.canvas.pack(fill="both", expand=True)

        # draw a object that we can move
        self.thing = self.canvas.create_rectangle(-20,-20,20,20, fill="red")

        # draw some random circles, so it's obvious when the canvas scrolls
        for i in range(100):
            x = random.randint(-300, 300)
            y = random.randint(-300, 300)
            radius = random.randint(10, 100)
            self.canvas.create_oval(x,y,x+radius, y+radius)

        # set bindings to move the thing around
        self.canvas.bind("<Left>",  lambda event: self.move(-5, 0))
        self.canvas.bind("<Right>", lambda event: self.move(5, 0))
        self.canvas.bind("<Up>",    lambda event: self.move(0,-5))
        self.canvas.bind("<Down>",  lambda event: self.move(0,5))
        self.canvas.bind("c",       lambda event: self.recenter())

        self.recenter()
        self.canvas.focus_set()

    def recenter(self):
        # center the viewport on the canvas
        # (because we know the window size, we know we want 
        # 50% of the window viewable)
        self.canvas.xview("moveto", "0.25")
        self.canvas.yview("moveto", "0.25")

    def make_visible(self):
        # determine the bounding box of the visible area of the screen
        (cx0,cy0) = (self.canvas.canvasx(0), self.canvas.canvasy(0))
        (cx1,cy1) = (self.canvas.canvasx(self.canvas.cget("width")), 
                     self.canvas.canvasy(self.canvas.cget("height")))

        # determine the bounding box of the thing to be made visible
        (x0,y0,x1,y1) = self.canvas.coords(self.thing)

        # determine how far off the screen the object is
        deltax = x0-cx0 if x0 <= cx0 else x1-cx1 if x1 >= cx1 else 0
        deltay = y0-cy0 if y0 <= cy0 else y1-cy1 if y1 >= cy1 else 0

        # scroll the canvas to make the item visible
        self.canvas.xview("scroll", int(deltax), "units")
        self.canvas.yview("scroll", int(deltay), "units")

    def move(self, deltax, deltay):
        # make sure we don't move beyond our scrollable region
        (x0,y0,x1,y1) = self.canvas.coords(self.thing)
        deltay = 0 if (deltay > 0 and y1 >= 400) else deltay
        deltay = 0 if (deltay < 0 and y0 <= -400) else deltay
        deltax = 0 if (deltax > 0 and x1 >= 400) else deltax
        deltax = 0 if (deltax < 0 and x0 <= -400) else deltax

        # move the item, then scroll it into view
        self.canvas.move(self.thing, deltax, deltay)
        self.make_visible()    

if __name__ == "__main__":
    root = tk.Tk()
    Example(root).pack(fill="both", expand=True)
    root.mainloop()

In [ ]:
from configparser import SectionProxy
import tkinter
import customtkinter

class PreferenceView(customtkinter.CTkFrame):
    WINDOW_WIDTH = 780
    def __init__(self, parent: customtkinter.CTkFrame, name: str, items: SectionProxy):
        super().__init__(master=parent)
        self.__items = items
        self.__name = name
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(len(self.__items.keys())+4, weight=2)
        
        self.header = customtkinter.CTkFrame(self, height=45, fg_color=("white", "gray45"))
        self.header.grid(row=0, column=0, columnspan=2, rowspan=2, sticky="nsew")
           
        self.text_label = customtkinter.CTkLabel(self.header, text=f"{self.__name} Preference", text_font=("Roboto Medium", -17))
        self.text_label.grid_rowconfigure(0, weight=1)
        self.text_label.grid(row=0, column=0, columnspan=2, rowspan=2, sticky="nsew", padx=10)


        for idx, item in enumerate(self.__items.keys()):
            self.sub_frame = customtkinter.CTkFrame(self)
            self.sub_frame.grid_rowconfigure(2, weight=1)
            self.sub_frame.grid_columnconfigure(2, weight=1)
            self.sub_frame.grid(row=idx+2, columnspan=2, rowspan=1, sticky="nsew")
            left_side = customtkinter.CTkLabel(master=self.sub_frame, text=item)
            left_side.grid(row=idx+2, column=1, columnspan=1, rowspan=1, sticky="ew", padx=5)
            customtkinter.CTkEntry(master=self.sub_frame, width=205, placeholder_text=self.__items[item]).grid(row=idx+2, column=2, columnspan=1, rowspan=1, sticky="e", padx=(10,10))


In [ ]:
# self.canvas = Canvas(master=self.frame_right, highlightthickness=0, bd=-2)
        # self.canvas.pack(fill=BOTH, expand=True)

        # self.canvas_scrollbar= ttk.Scrollbar(self.frame_right, orient=VERTICAL, command=self.canvas.yview)
        # self.canvas_scrollbar.pack(side=RIGHT, fill="y", expand=False)

        # self.canvas.configure(yscrollcommand=self.canvas_scrollbar.set)
        # self.canvas.bind('<Configure>', lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all")))
        # def _on_mouse_wheel(event):
        #     self.canvas.yview_scroll(-1 * int((event.delta / 120)), "units")
        # self.canvas.bind_all("<MouseWheel>", _on_mouse_wheel)
        # self.canvas.yview_moveto(0)

        # self.list_frame = customtkinter.CTkFrame(self.canvas)

        # self.canvas_scrollbar.lift(self.list_frame)
        # self.list_frame.pack(expand=True)
        # self.list_frame.bind('<Configure>', self._configure_window)
        # self.list_frame.bind('<Enter>', self._bound_to_mousewheel)
        # self.list_frame.bind('<Leave>', self._unbound_to_mousewheel)
        
        # for key in self.__client_data.pref.keys():
        #     if list(self.__client_data.pref[key].keys()):
        #         PreferenceView(parent=self.list_frame, name=key, items=self.__client_data.pref[key]).pack(fill=BOTH, expand=True)

        # self.list_frame.pack(fill=BOTH, expand=True)
        # self.interior_id = self.canvas.create_window((0,0), window=self.list_frame, anchor="nw")
        # self.canvas.pack(fill=BOTH, expand=True)
        
        
        # self.button_2 = customtkinter.CTkButton(master=self.frame_left,
        #                                         text="Button2",
        #                                         command=self.button_event)
        # self.button_2.grid(row=3, column=0, pady=10, padx=20)

        # self.button_3 = customtkinter.CTkButton(master=self.frame_left,
        #                                         text="Button3",
        #                                         command=self.button_event)
        # self.button_3.grid(row=4, column=0, pady=10, padx=20)

        # self.label_mode = customtkinter.CTkLabel(master=self.frame_left, text="Appearance Mode:")
        # self.label_mode.grid(row=9, column=0, pady=0, padx=20, sticky="w")

        # self.optionmenu_1 = customtkinter.CTkOptionMenu(master=self.frame_left,
        #                                                 values=["Light", "Dark", "System"],
        #                                                 command=self.change_appearance_mode)
        # self.optionmenu_1.grid(row=10, column=0, pady=10, padx=20, sticky="w")